In [1]:
import sys

sys.path.append("../")

In [2]:
cfg_path = "../denseclip_configs/denseclip_fpn_res50_512x512_80k.py"

In [3]:
from mmengine.config import Config
from mmseg.models import build_segmentor

/home/maverick/Projects/ml-scratchpad/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


please install Colossal-AI from https://www.colossalai.org/download or from source


/home/maverick/Projects/ml-scratchpad/notebooks/../colossalai/pipeline/schedule/_utils.py:19: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _register_pytree_node(OrderedDict, _odict_flatten, _odict_unflatten)
/home/maverick/Projects/ml-scratchpad/.venv/lib/python3.12/site-packages/torch/utils/_pytree.py:300: UserWarning: <class 'collections.OrderedDict'> is already registered as pytree node. Overwriting the previous registration.
  warnings.warn(
/home/maverick/Projects/ml-scratchpad/notebooks/../colossalai/shardformer/layer/normalization.py:45: UserWarning: Please install apex from source (https://github.com/NVIDIA/apex) to use the fused layernorm kernel
  warnings.warn("Please install apex from source (https://github.com/NVIDIA/apex) to use the fused layernorm kernel")


In [4]:
import src.models.components.denseclip

In [5]:
cfg = Config.fromfile(cfg_path)

In [6]:
cfg

Config (path: ../denseclip_configs/denseclip_fpn_res50_512x512_80k.py): {'norm_cfg': {'type': 'SyncBN', 'requires_grad': True}, 'model': {'type': 'DenseCLIP', 'pretrained': 'pretrained/RN50.pt', 'context_length': 5, 'backbone': {'type': 'CLIPResNetWithAttention', 'layers': [3, 4, 6, 3], 'style': 'pytorch', 'output_dim': 1024, 'input_resolution': 512}, 'text_encoder': {'type': 'CLIPTextContextEncoder', 'context_length': 13, 'style': 'pytorch', 'embed_dim': 1024, 'transformer_width': 512, 'transformer_heads': 8, 'transformer_layers': 12}, 'context_decoder': {'type': 'ContextDecoder', 'context_length': 16, 'transformer_width': 256, 'transformer_heads': 4, 'transformer_layers': 3, 'visual_dim': 1024, 'dropout': 0.1, 'outdim': 1024, 'style': 'pytorch'}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2198], 'out_channels': 256, 'num_outs': 4}, 'decode_head': {'type': 'FPNHead', 'in_channels': [256, 256, 256, 256], 'in_index': [0, 1, 2, 3], 'feature_strides': [4, 8, 16, 32], 'channel

In [7]:
cfg.model.pretrained = None
cfg.data.test.test_mode = True
cfg.model.train_cfg = None
cfg.model.class_names = ['wall', 'building', 'sky', 'floor', 'tree', 'ceiling', 'road',
                 'bed ', 'windowpane', 'grass', 'cabinet', 'sidewalk',
                 'person', 'earth', 'door', 'table', 'mountain', 'plant',
                 'curtain', 'chair', 'car', 'water', 'painting', 'sofa',
                 'shelf', 'house', 'sea', 'mirror', 'rug', 'field', 'armchair',
                 'seat', 'fence', 'desk', 'rock', 'wardrobe', 'lamp',
                 'bathtub', 'railing', 'cushion', 'base', 'box', 'column',
                 'signboard', 'chest of drawers', 'counter', 'sand', 'sink',
                 'skyscraper', 'fireplace', 'refrigerator', 'grandstand',
                 'path', 'stairs', 'runway', 'case', 'pool table', 'pillow',
                 'screen door', 'stairway', 'river', 'bridge', 'bookcase',
                 'blind', 'coffee table', 'toilet', 'flower', 'book', 'hill',
                 'bench', 'countertop', 'stove', 'palm', 'kitchen island',
                 'computer', 'swivel chair', 'boat', 'bar', 'arcade machine',
                 'hovel', 'bus', 'towel', 'light', 'truck', 'tower',
                 'chandelier', 'awning', 'streetlight', 'booth',
                 'television receiver', 'airplane', 'dirt track', 'apparel',
                 'pole', 'land', 'bannister', 'escalator', 'ottoman', 'bottle',
                 'buffet', 'poster', 'stage', 'van', 'ship', 'fountain',
                 'conveyer belt', 'canopy', 'washer', 'plaything',
                 'swimming pool', 'stool', 'barrel', 'basket', 'waterfall',
                 'tent', 'bag', 'minibike', 'cradle', 'oven', 'ball', 'food',
                 'step', 'tank', 'trade name', 'microwave', 'pot', 'animal',
                 'bicycle', 'lake', 'dishwasher', 'screen', 'blanket',
                 'sculpture', 'hood', 'sconce', 'vase', 'traffic light',
                 'tray', 'ashcan', 'fan', 'pier', 'crt screen', 'plate',
                 'monitor', 'bulletin board', 'shower', 'radiator', 'glass',
                 'clock', 'flag']

In [8]:
cfg.model

{'type': 'DenseCLIP',
 'pretrained': None,
 'context_length': 5,
 'backbone': {'type': 'CLIPResNetWithAttention',
  'layers': [3, 4, 6, 3],
  'style': 'pytorch',
  'output_dim': 1024,
  'input_resolution': 512},
 'text_encoder': {'type': 'CLIPTextContextEncoder',
  'context_length': 13,
  'style': 'pytorch',
  'embed_dim': 1024,
  'transformer_width': 512,
  'transformer_heads': 8,
  'transformer_layers': 12},
 'context_decoder': {'type': 'ContextDecoder',
  'context_length': 16,
  'transformer_width': 256,
  'transformer_heads': 4,
  'transformer_layers': 3,
  'visual_dim': 1024,
  'dropout': 0.1,
  'outdim': 1024,
  'style': 'pytorch'},
 'neck': {'type': 'FPN',
  'in_channels': [256, 512, 1024, 2198],
  'out_channels': 256,
  'num_outs': 4},
 'decode_head': {'type': 'FPNHead',
  'in_channels': [256, 256, 256, 256],
  'in_index': [0, 1, 2, 3],
  'feature_strides': [4, 8, 16, 32],
  'channels': 256,
  'dropout_ratio': 0.1,
  'num_classes': 150,
  'norm_cfg': {'type': 'SyncBN', 'require

In [9]:
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))

/home/maverick/Projects/ml-scratchpad/notebooks/../mmseg/models/builder.py:15: UserWarning: ``build_backbone`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn(
/home/maverick/Projects/ml-scratchpad/notebooks/../mmseg/models/builder.py:24: UserWarning: ``build_neck`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn(
/home/maverick/Projects/ml-scratchpad/notebooks/../mmseg/models/builder.py:33: UserWarning: ``build_head`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn(
/home/maverick/Projects/ml-scratchpad/notebooks/../mmseg/models/builder.py:42: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn(
/home/maverick/Projects/ml-scratchpad/notebooks/../mmseg/models/losses/cross_entropy_loss.py:254: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss o

In [10]:
model

DenseCLIP(
  (data_preprocessor): BaseDataPreprocessor()
  (backbone): CLIPResNetWithAttention(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=

In [11]:
model.texts

tensor([[  512,  2571,   513,     0,     0],
        [  512,  2309,   513,     0,     0],
        [  512,  2392,   513,     0,     0],
        [  512,  4127,   513,     0,     0],
        [  512,  2679,   513,     0,     0],
        [  512, 12376,   513,     0,     0],
        [  512,  1761,   513,     0,     0],
        [  512,  2724,   513,     0,     0],
        [  512, 26677, 27472,   513,     0],
        [  512,  5924,   513,     0,     0],
        [  512,  9939,   513,     0,     0],
        [  512, 23280,   513,     0,     0],
        [  512,  2535,   513,     0,     0],
        [  512,  3477,   513,     0,     0],
        [  512,  2491,   513,     0,     0],
        [  512,  2177,   513,     0,     0],
        [  512,  3967,   513,     0,     0],
        [  512,  3914,   513,     0,     0],
        [  512, 17225,   513,     0,     0],
        [  512,  4271,   513,     0,     0],
        [  512,  1617,   513,     0,     0],
        [  512,  1575,   513,     0,     0],
        [ 